<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/SEAL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Continual-Intelligence/SEAL/tree/main

In [ ]:
!git clone https://github.com/Continual-Intelligence/SEAL.git
%cd SEAL
!pip install -r requirements.txt -q
!pip install colab-env -q
import colab_env

In [5]:
!nvidia-smi

Thu Jul  3 10:29:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

In [ ]:
!sh /content/SEAL/few-shot/launch.sh

In [6]:
DATA_DIR='/content/SEAL/few-shot/data'
!python /content/SEAL/few-shot/self-edit.py  \
    --experiment_name=training_set_iteration_1 \
    --challenge_file={DATA_DIR}/arc-agi_training_challenges_filtered_1B_training_set.json \
    --solution_file={DATA_DIR}/arc-agi_training_solutions_filtered_1B_training_set.json \
    --model_name=meta-llama/Llama-3.2-1B-Instruct \
    --n_tasks=1 \
    --n_self_edits_per_task=15

2025-07-03 10:36:08.479950: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-03 10:36:08.497782: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751538968.519379   62479 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751538968.525991   62479 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-03 10:36:08.548080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
!python /content/SEAL/few-shot/ttt.py

In [ ]:
DATA_DIR='/content/SEAL/few-shot/data'
TTI_DIR='/content/SEAL/loras/self-edit/training_set_iteration_1'
LORA_DIR='/content/SEAL/loras/self-edit/training_set_iteration_1/8d5021e8/14'

!python /content/SEAL/few-shot/eval-self-edits.py  \
    --experiment_folder={TTI_DIR} \
    --pretrained_checkpoint=meta-llama/Llama-3.2-1B-Instruct \
    --lora_checkpoints_folder={LORA_DIR} \
    --temperature=0 \
    --n_sample=1 \
    --data_file=/content/SEAL/few-shot/data/arc-agi_training_challenges_filtered_1B_training_set.json \
    --solution_file=/content/SEAL/few-shot/data/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=11 \
    --n_self_edits=15

In [ ]:
!python /content/SEAL/few-shot/BC-self-edit.py \
    --configs_and_indices=/content/SEAL/loras/self-edit/training_set_iteration_1/final_configs_and_indices.json \
    --results=/content/SEAL/few-shot/final_results.json \
    --model_name=meta-llama/Llama-3.2-1B-Instruct \
    --lora_rank=16 \
    --lora_alpha=16 \
    --num_train_epochs=8 \
    --per_device_train_batch_size=5 \
    --gradient_accumulation_steps=1 \
    --learning_rate=5e-5

In [ ]:
!python /content/SEAL/few-shot/self-edit.py  \
    --experiment_name=eval_RL_iteration_1_8_epoch \
    --challenge_file={DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file={DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --model_name={LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1_8_epoch \
    --n_tasks=10 \
    --n_self_edits_per_task=5

In [ ]:
!python /content/SEAL/few-shot/eval-self-edits.py \
    --experiment_folder={TTI_DIR}/eval_set_RL_iteration_1_8_epoch \
    --pretrained_checkpoint={LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1_8_epoch \
    --lora_checkpoints_folder={LORA_DIR}/self-edit/eval_RL_iteration_1_8_epoch \
    --temperature=0 \
    --n_sample=1 \
    --data_file=${DATA_DIR}/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=${DATA_DIR}/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=9 \
    --n_self_edits=5

In [ ]:
!python /content/SEAL/few-shot/eval-self-edits-baseline.py \
    --experiment_folder=/content/SEAL/tti/eval_base_model \
    --pretrained_checkpoint=meta-llama/Llama-3.2-1B-Instruct \
    --lora_checkpoints_folder=${LORA_DIR}/self-edit/eval_RL_iteration_1_8_epoch \
    --temperature=0 \
    --n_sample=1 \
    --data_file=/content/SEAL/few-shot/data/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=/content/SEAL/few-shot/data/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --max_lora_rank=128 \
    --include_n=1 \
    --new_format \
    --num_examples=9

In [ ]:
!python /content/SEAL/few-shot/self-edit.py \
    --experiment_name=eval_RL_iteration_1 \
    --challenge_file=/content/SEAL/few-shot/data/arc-agi_evaluation_challenges_filtered_1B_eval_set.json \
    --solution_file=/content/SEAL/few-shot/data/arc-agi_evaluation_solutions_filtered_1B_eval_set.json \
    --model_name=${LORA_DIR}/self-edit/training_set_iteration_1/RL_trained_model_iteration_1 \
    --n_tasks=10 \
    --n_self_edits_per_task=5